In [ ]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from dotenv import load_dotenv
from datetime import datetime

In [14]:
load_dotenv('../.env')

True

In [16]:
TASK_NAME = "source_parsing_v0"

def get_run_id():
    return os.getenv('RUNID')

RUNID = get_run_id()

INPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/{RUNID}/{TASK_NAME}/BATCHFILES"

RUN_TIME = datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M:%S')

OUTPUT_DATA_PATH = f"../local_tests_data/azure_openai_batch_processing_files/{RUNID}/{TASK_NAME}/BATCHID"

os.makedirs(OUTPUT_DATA_PATH, exist_ok=True)

print(f"Run ID: {RUNID} at {RUN_TIME}")

Run ID: RUNID_2 at 2025-05-30 11:08:25


In [17]:
AZURE_OPENAI_API_KEY=os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT=os.getenv('AZURE_OPENAI_ENDPOINT')
client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [18]:
def get_batchfiles():
    batchfiles = []
    for filename in os.listdir(INPUT_DATA_PATH):
        if filename.endswith('.jsonl'):
            b = {
                'path': os.path.join(INPUT_DATA_PATH, filename),
                'id' : filename.split('.')[0].split('_')[-1],
            }
            batchfiles.append(b)
    return sorted(batchfiles)

batchfiles = get_batchfiles()

In [19]:
def save_batchid(i, batch_id):
    batchid_filename = f"{RUNID}--{TASK_NAME}_BATCHID_{i}.txt"
    with open(os.path.join(OUTPUT_DATA_PATH, batchid_filename), "w") as f:
        f.write(batch_id)

In [20]:

for bathcfile in batchfiles:

  i = bathcfile['id']
  path = bathcfile['path']

  print("Submitting {}".format(path))

  file = client.files.create(
    file=open(path, "rb"),
    purpose="batch"
  )

  print(file.model_dump_json(indent=2))
  file_id = file.id

  # Submit a batch job with the file
  batch_response = client.batches.create(
      input_file_id=file_id,
      endpoint="/chat/completions",
      completion_window="24h",
  )

  # Save batch ID for later use
  batch_id = batch_response.id

  print(batch_response.model_dump_json(indent=2)) 

  save_batchid(i, batch_id)
  print(f"Batch ID for file {i}: {batch_id}")
  print("Batch job submitted successfully.")
  print("  ")

Submitting ../local_tests_data/azure_openai_batch_processing_files/RUNID_2/source_parsing_v0/BATCHFILES/RUNID_2--source_parsing_v0_BATCHFILE_0.jsonl
{
  "id": "file-8c0363975be24d6680b722bb5eec6f4e",
  "bytes": 113490,
  "created_at": 1748596109,
  "filename": "RUNID_2--source_parsing_v0_BATCHFILE_0.jsonl",
  "object": "file",
  "purpose": "batch",
  "status": "processed",
  "expires_at": null,
  "status_details": null
}
{
  "id": "batch_80f38070-5b3f-4b55-846b-448b0561715a",
  "completion_window": "24h",
  "created_at": 1748596112,
  "endpoint": "/chat/completions",
  "input_file_id": "file-8c0363975be24d6680b722bb5eec6f4e",
  "object": "batch",
  "status": "validating",
  "cancelled_at": null,
  "cancelling_at": null,
  "completed_at": null,
  "error_file_id": "",
  "errors": null,
  "expired_at": null,
  "expires_at": 1748682510,
  "failed_at": null,
  "finalizing_at": null,
  "in_progress_at": null,
  "metadata": null,
  "output_file_id": "",
  "request_counts": {
    "completed": 